## 深度学习特征

提取CT、MRI、内镜、Xray等影像数据的深度学习特征。

### Onekey步骤

1. 将待提取的数据转化成nii(nii.gz)，需要使用到OKT-convert2nii工具。
2. 获取到指定目录的所有图像数据。
3. 选择要提取什么样的模型的深度学习特征，目前Onekey支持ResNet3d深度学习模型。（可以考虑使用Onekey进行迁移学习）
  > 只支持ResNet3d，是因为目前仅有resnet存在预训练的模型。
4. 提取特征，保存特征文件。

### 使用crop max roi工具保存3dnii数据

参数axis_3d > 2 即可使用保存roi的3d数据，不进行最大面积截断

### 获取待提取特征的NII数据

提供两种批量处理的模式：
1. 目录模式，提取指定目录下的所有jpg文件的特征。
2. 文件模式，待提取的数据存储在文件中，每行一个样本。

当然也可以在最后自己指定手动提取指定若干文件。

In [ ]:
import os
from onekey_algo import OnekeyDS as okds
from onekey_algo import get_param_in_cwd
# 目录模式
mydir = get_param_in_cwd('data_pattern')
directory = os.path.expanduser(mydir)
test_samples = [os.path.join(directory, p) 
                for p in os.listdir(directory) if p.endswith('.nii') or p.endswith('.nii.gz')]
test_samples

## 确定提取特征

通过关键词获取要提取那一层的特征。

### 支持的模型名称

模型名称替换代码中的 `model_name`变量的值。

| **模型系列** | **模型名称**                                                 |
| ------------ | ------------------------------------------------------------ |
| ResNet       | resnet10, resnet18, resnet34, resnet50, resnet101, resnet152, resnet200 |

In [ ]:
from monai.data import ImageDataset
from torch.utils.data import DataLoader
from onekey_algo.custom.components.comp2 import extract, init_from_onekey3d

model_name = get_param_in_cwd('sel_dl_model_name')
model_root = os.path.join(get_param_in_cwd('model_root'), model_name)
model, transformer, device = init_from_onekey3d(os.path.join(model_root, 'viz'))
for n, m in model.named_modules():
    print('Feature name:', n, "|| Module:", m)

## 提取特征

`Feature name:` 之后的名称为要提取的特征名，例如`layer3.0.conv2`, 一般深度学习特征提取最后一层，例如`avgpool`

In [ ]:
import torch
from functools import partial
from onekey_algo.custom.components.comp2 import extract3d, print_feature_hook, reg_hook_on_module
from monai.data import ImageDataset
from torch.utils.data import DataLoader

feature_name = 'features.adaptive_avg_pool3d'
os.makedirs('features', exist_ok=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with open(f'features/{model_name}_features.csv', 'w') as outfile:
    hook = partial(print_feature_hook, fp=outfile)
    find_num = reg_hook_on_module(feature_name, model, hook)
    val_ds = ImageDataset(image_files=test_samples, transform=transformer)
    # create a validation data loader
    val_loader = DataLoader(val_ds, batch_size=1, num_workers=0)
    
    results = extract3d(val_loader, test_samples, model, device, fp=outfile)

### 读取数据

In [ ]:
import pandas as pd
features = pd.read_csv(f'features/{model_name}_features.csv', header=None)
features.columns=['ID'] + [f"DL_{i}" for i in range(features.shape[1] - 1)]
features.to_csv(f'features/{model_name}_features.csv', index=False)
features

### 深度特征压缩

深度学习特征压缩，注意压缩到的维度需要小于样本数

```python
def compress_df_feature(features: pd.DataFrame, dim: int, not_compress: Union[str, List[str]] = None,
                        prefix='') -> pd.DataFrame:
    """
    压缩深度学习特征
    Args:
        features: 特征DataFrame
        dim: 需要压缩到的维度，此值需要小于样本数
        not_compress: 不进行压缩的列。
        prefix: 所有特征的前缀。

    Returns:

    """
```

In [ ]:
from onekey_algo.custom.components.comp1 import compress_df_feature

cm_features = compress_df_feature(features=features, dim=32, prefix='DL_', not_compress='ID')
cm_features.to_csv(f'features/{model_name}_compress_features.csv', header=True, index=False)